This runs in 8 minutes on 1xA6000

In [ ]:
%pip install sentence_transformers

In [2]:

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("Salesforce/SFR-Embedding-Mistral")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
import os
import re
from datetime import datetime

PROVIDERS = {
    "google": {
        "raw": "Takeout/My Activity/Search",
        "parsed": "google/search_history",
        "summary": "google/search_history_summary",
        "context": "",
        "interests": ""
    }
}


def get_filenames(
    kind="parsed", start_date=None, end_date=None, provider="google"
):
    directory = os.path.join(kind, PROVIDERS[provider][kind])
    if start_date is not None:
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if end_date is not None:
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    file_pattern = r"^(\d{4}-\d{2}-\d{2})\.(csv|json)$"

    def is_date_in_range(file_date):
        if start_date is None and end_date is None:
            return True
        else:
            return start_date <= datetime.strptime(file_date, "%Y-%m-%d") <= end_date

    filenames = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            match = re.match(file_pattern, file)
            if match:
                file_date = match.groups()[0]
                if is_date_in_range(file_date):
                    filenames.append(os.path.join(root, file))

    return filenames

In [5]:
# load the results
import json
from collections import defaultdict

results_dict = defaultdict(list)

for filename in get_filenames(kind="interests"):
    date = filename.split("/")[-1].split(".")[0]
    results_dict[date] = list(map(lambda x: x.strip(), json.load(open(filename))))

In [ ]:
# craete a dataframe with the embeddings and interests for each date

import pandas as pd
from tqdm.notebook import tqdm
import numpy as np


df = pd.DataFrame(columns=["date", "interest", "embedding"])

for date, interests in tqdm(results_dict.items(), smoothing=0):
    np.save(f"embeddings/{date}.npy", model.encode(interests))
